<a href="https://colab.research.google.com/github/Dr-Future/CSC413-Project-Winter2023/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import and util code

In [1]:
%matplotlib inline

import os
import pdb
import argparse
import pickle as pkl
from pathlib import Path

from collections import defaultdict

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

from six.moves.urllib.request import urlretrieve
import tarfile
import pickle
import sys

import pandas as pd
import seaborn as sns



In [ ]:
# For KNN and logistic regression
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Data Laundry

In [24]:
import re
def remove_newlines(string):
    # Remove '\r', '\n', and '\t' characters using regex
    return re.sub(r'[\r\n\t]', '', string)

In [38]:
def get_digit(s):
  num_str = ""
  for c in s:
      if c.isdigit():
          num_str += c

  return int(num_str)


In [59]:

import chardet

#load_data

folder_path = '/content/gdrive/My Drive/CSC413 project/test_data' # Replace with the path to your folder containing subfolders with text files

# Define a dictionary to store the file contents
data_sol = {} #(key=id, value = string)
data_ast = {} #(key=id, value = string)
vulnerability_classes = {} #(key=id, value = arrayofvoltypes)
vulnerability_class_names = []


# Loop over each subfolder, and then each file in each subfolder
for subfolder_name in os.listdir(folder_path):
  subfolder_path = os.path.join(folder_path, subfolder_name)
  if os.path.isdir(subfolder_path):
    vulnerability_class_names.append(subfolder_name)
    for file_name in os.listdir(subfolder_path):
      file_path = os.path.join(subfolder_path, file_name)
      if os.path.isfile(file_path) and file_name.endswith('.sol'):
        file_id = get_digit(file_name)
        if file_id in vulnerability_classes:
          vulnerability_classes[file_id].append(subfolder_name)
        else:
          vulnerability_classes[file_id] = [subfolder_name]
        if not(file_id in data_sol):
          with open(file_path, 'r') as f:
            contents = f.read()
            data_sol[file_id] = remove_newlines(contents)
      if os.path.isdir(file_path):
        for ast_file_name in os.listdir(file_path):
           ast_file_path = os.path.join(file_path, ast_file_name)
           ast_file_id = get_digit(ast_file_name)
           if os.path.isfile(ast_file_path) and ast_file_name.endswith('.json') and  not(ast_file_id in data_ast):
             with open(ast_file_path, 'rb') as f:
              raw_data = f.read()
              encoding = chardet.detect(raw_data)['encoding']
              contents = raw_data.decode(encoding)
              data_ast[ast_file_id] = remove_newlines(contents)

              folder_path = '/content/gdrive/My Drive/CSC413 project/test_data' # Replace with the path to your folder containing subfolders with text files

# # Define a dictionary to store the file contents
# data_sol = {}
# data_ast = {}
# vulnerability_classes = []


# # Loop over each subfolder, and then each file in each subfolder
# for subfolder_name in os.listdir(folder_path):
#   subfolder_path = os.path.join(folder_path, subfolder_name)
#   if os.path.isdir(subfolder_path):
#     data_sol[subfolder_name] = []
#     vulnerability_classes.append(subfolder_name)
#     for file_name in os.listdir(subfolder_path):
#       file_path = os.path.join(subfolder_path, file_name)
#       if os.path.isfile(file_path) and file_name.endswith('.sol'):
#         with open(file_path, 'r') as f:
#           contents = f.read()
#           data_sol[subfolder_name].append(contents)
#       if os.path.isdir(file_path):
#         data_ast[subfolder_name] = []
#         for ast_file_name in os.listdir(file_path):
#            ast_file_path = os.path.join(file_path, ast_file_name)
#            if os.path.isfile(ast_file_path) and ast_file_name.endswith('.json'):
#              with open(ast_file_path, 'rb') as f:
#               raw_data = f.read()
#               encoding = chardet.detect(raw_data)['encoding']
#               contents = raw_data.decode(encoding)
#               data_ast[subfolder_name].append(remove_newlines(contents))

          



In [45]:
vulnerability_classes

{0: ['block_number_dependency_BN'],
 84: ['block_number_dependency_BN'],
 103: ['block_number_dependency_BN'],
 112: ['block_number_dependency_BN'],
 129: ['block_number_dependency_BN'],
 135: ['block_number_dependency_BN'],
 3521: ['dangerous_delegatecall_DE', 'ether_frozen_EF'],
 5854: ['dangerous_delegatecall_DE'],
 10708: ['dangerous_delegatecall_DE'],
 2004: ['dangerous_delegatecall_DE', 'ether_frozen_EF'],
 4617: ['dangerous_delegatecall_DE'],
 3972: ['dangerous_delegatecall_DE'],
 43311: ['ether_frozen_EF'],
 10726: ['ether_frozen_EF'],
 44311: ['ether_frozen_EF']}

In [61]:
vulnerability_class_names

['block_number_dependency_BN', 'dangerous_delegatecall_DE', 'ether_frozen_EF']

In [60]:
vulnerability_vector = {id: [1 if class_name in vulnerability_classes[id] else 0 for class_name in vulnerability_class_names] for id in data_sol}
vulnerability_vector

{0: [1, 0, 0],
 84: [1, 0, 0],
 103: [1, 0, 0],
 112: [1, 0, 0],
 129: [1, 0, 0],
 135: [1, 0, 0],
 3521: [0, 1, 1],
 5854: [0, 1, 0],
 10708: [0, 1, 0],
 2004: [0, 1, 1],
 4617: [0, 1, 0],
 3972: [0, 1, 0],
 43311: [0, 0, 1],
 10726: [0, 0, 1],
 44311: [0, 0, 1]}

In [ ]:
# Get the data here
# Maybe write a function to return processed data
x_train = #...
y_train = #...
x_test = #...
y_test = #...

# K Nearest Neighbour

In [ ]:
def train_knn(k: int):
  # Load the Iris dataset (toy data for debuging only)
  iris = datasets.load_iris()
  x, y = iris.data, iris.target

  # Split the dataset into training and testing sets
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

  # Create a KNN classifier with k neighbors
  knn = KNeighborsClassifier(n_neighbors = k)

  # Train the classifier with the training data
  knn.fit(x_train, y_train)

  # Make predictions on the training data
  y_train_pred = knn.predict(x_train)

  # Evaluate the model's performance on the training set
  print("Performance on the training set:")
  print("Confusion Matrix:")
  print(confusion_matrix(y_train, y_train_pred))
  print("\nClassification Report:")
  print(classification_report(y_train, y_train_pred))

  # Make predictions on the testing data
  y_test_pred = knn.predict(x_test)

  # Evaluate the model's performance on the testing set
  print("\nPerformance on the testing set:")
  print("Confusion Matrix:")
  print(confusion_matrix(y_test, y_test_pred))
  print("\nClassification Report:")
  print(classification_report(y_test, y_test_pred))

In [ ]:
train_knn(k = 3)

Performance on the training set:
Confusion Matrix:
[[40  0  0]
 [ 0 38  3]
 [ 0  3 36]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       0.93      0.93      0.93        41
           2       0.92      0.92      0.92        39

    accuracy                           0.95       120
   macro avg       0.95      0.95      0.95       120
weighted avg       0.95      0.95      0.95       120


Performance on the testing set:
Confusion Matrix:
[[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



# Stochastic Gradient Descent

In [ ]:
def train_SGD():
  # Load the Iris dataset (toy data for debuging only)
  iris = datasets.load_iris()
  x, y = iris.data, iris.target

  # Split the dataset into training and testing sets
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

  # Create an SGD classifier
  sgd_clf = SGDClassifier()

  # Train the model with the training data
  sgd_clf.fit(x_train, y_train)

  # Make predictions on the training data
  y_train_pred = sgd_clf.predict(x_train)

  # Evaluate the model's performance on the training set
  print("Performance on the training set:")
  print("Confusion Matrix:")
  print(confusion_matrix(y_train, y_train_pred))
  print("\nClassification Report:")
  print(classification_report(y_train, y_train_pred))

  # Make predictions on the testing data
  y_test_pred = sgd_clf.predict(x_test)

  # Evaluate the model's performance on the testing set
  print("\nPerformance on the testing set:")
  print("Confusion Matrix:")
  print(confusion_matrix(y_test, y_test_pred))
  print("\nClassification Report:")
  print(classification_report(y_test, y_test_pred))

In [ ]:
train_SGD()

Performance on the training set:
Confusion Matrix:
[[40  0  0]
 [31  0 10]
 [ 0  0 39]]

Classification Report:
              precision    recall  f1-score   support

           0       0.56      1.00      0.72        40
           1       0.00      0.00      0.00        41
           2       0.80      1.00      0.89        39

    accuracy                           0.66       120
   macro avg       0.45      0.67      0.54       120
weighted avg       0.45      0.66      0.53       120


Performance on the testing set:
Confusion Matrix:
[[10  0  0]
 [ 5  0  4]
 [ 0  0 11]]

Classification Report:
              precision    recall  f1-score   support

           0       0.67      1.00      0.80        10
           1       0.00      0.00      0.00         9
           2       0.73      1.00      0.85        11

    accuracy                           0.70        30
   macro avg       0.47      0.67      0.55        30
weighted avg       0.49      0.70      0.58        30



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classificat

# Transformer

## Utility code

In [ ]:
def save_loss_plot(train_losses, val_losses, opts):
    """Saves a plot of the training and validation loss curves."""
    plt.figure()
    plt.plot(range(len(train_losses)), train_losses)
    plt.plot(range(len(val_losses)), val_losses)
    plt.title("BS={}, nhid={}".format(opts.batch_size, opts.hidden_size), fontsize=20)
    plt.xlabel("Epochs", fontsize=16)
    plt.ylabel("Loss", fontsize=16)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.tight_layout()
    plt.savefig(os.path.join(opts.checkpoint_path, "loss_plot.pdf"))
    plt.close()

## Training and evaluating

In [ ]:
def compute_loss(data_dict, encoder, decoder, idx_dict, criterion, optimizer, opts):
    """Train/Evaluate the model on a dataset.

    Arguments:
        data_dict: The validation/test word pairs, organized by source and target lengths.
        encoder: An encoder model to produce annotations for each step of the input sequence.
        decoder: A decoder model (with or without attention) to generate output tokens.
        idx_dict: Contains char-to-index and index-to-char mappings, and start & end token indexes.
        criterion: Used to compute the CrossEntropyLoss for each decoder output.
        optimizer: Train the weights if an optimizer is given. None if only evaluate the model.
        opts: The command-line arguments.

    Returns:
        mean_loss: The average loss over all batches from data_dict.
    """
    start_token = idx_dict["start_token"]
    end_token = idx_dict["end_token"]
    char_to_index = idx_dict["char_to_index"]

    losses = []
    for key in data_dict:
        input_strings, target_strings = zip(*data_dict[key])
        input_tensors = [
            torch.LongTensor(string_to_index_list(s, char_to_index, end_token))
            for s in input_strings
        ]
        target_tensors = [
            torch.LongTensor(string_to_index_list(s, char_to_index, end_token))
            for s in target_strings
        ]

        num_tensors = len(input_tensors)
        num_batches = int(np.ceil(num_tensors / float(opts.batch_size)))

        for i in range(num_batches):

            start = i * opts.batch_size
            end = start + opts.batch_size

            inputs = to_var(torch.stack(input_tensors[start:end]), opts.cuda)
            targets = to_var(torch.stack(target_tensors[start:end]), opts.cuda)

            # The batch size may be different in each epoch
            BS = inputs.size(0)

            encoder_annotations, encoder_hidden = encoder(inputs)

            # The last hidden state of the encoder becomes the first hidden state of the decoder
            decoder_hidden = encoder_hidden

            start_vector = (
                torch.ones(BS).long().unsqueeze(1) * start_token
            )  # BS x 1 --> 16x1  CHECKED
            decoder_input = to_var(start_vector, opts.cuda)  # BS x 1 --> 16x1  CHECKED

            loss = 0.0

            seq_len = targets.size(1)  # Gets seq_len from BS x seq_len

            decoder_inputs = torch.cat(
                [decoder_input, targets[:, 0:-1]], dim=1
            )  # Gets decoder inputs by shifting the targets to the right

            decoder_outputs, attention_weights = decoder(
                decoder_inputs, encoder_annotations, decoder_hidden
            )
            decoder_outputs_flatten = decoder_outputs.view(-1, decoder_outputs.size(2))
            targets_flatten = targets.view(-1)

            loss = criterion(decoder_outputs_flatten, targets_flatten)

            losses.append(loss.item())

            ## training if an optimizer is provided
            if optimizer:
                # Zero gradients
                optimizer.zero_grad()
                # Compute gradients
                loss.backward()
                # Update the parameters of the encoder and decoder
                optimizer.step()

    return losses


def training_loop(
    train_dict, val_dict, idx_dict, encoder, decoder, criterion, optimizer, opts
):
    """Runs the main training loop; evaluates the model on the val set every epoch.
        * Prints training and val loss each epoch.
        * Prints qualitative translation results each epoch using TEST_SENTENCE
        * Saves an attention map for TEST_WORD_ATTN each epoch
        * Returns loss curves for comparison

    Arguments:
        train_dict: The training word pairs, organized by source and target lengths.
        val_dict: The validation word pairs, organized by source and target lengths.
        idx_dict: Contains char-to-index and index-to-char mappings, and start & end token indexes.
        encoder: An encoder model to produce annotations for each step of the input sequence.
        decoder: A decoder model (with or without attention) to generate output tokens.
        criterion: Used to compute the CrossEntropyLoss for each decoder output.
        optimizer: Implements a step rule to update the parameters of the encoder and decoder.
        opts: The command-line arguments.

    Returns:
        losses: Lists containing training and validation loss curves.
    """
    loss_log = open(os.path.join(opts.checkpoint_path, "loss_log.txt"), "w")

    best_val_loss = 1e6
    train_losses = []
    val_losses = []

    mean_train_losses = []
    mean_val_losses = []

    early_stopping_counter = 0

    for epoch in range(opts.nepochs):

        optimizer.param_groups[0]["lr"] *= opts.lr_decay

        train_loss = compute_loss(
            train_dict, encoder, decoder, idx_dict, criterion, optimizer, opts
        )
        val_loss = compute_loss(
            val_dict, encoder, decoder, idx_dict, criterion, None, opts
        )

        mean_train_loss = np.mean(train_loss)
        mean_val_loss = np.mean(val_loss)

        if mean_val_loss < best_val_loss:
            best_val_loss = mean_val_loss
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1

        if early_stopping_counter > opts.early_stopping_patience:
            print(
                "Validation loss has not improved in {} epochs, stopping early".format(
                    opts.early_stopping_patience
                )
            )
            print("Obtained lowest validation loss of: {}".format(best_val_loss))
            return (train_losses, mean_val_losses)

        print(
            "Epoch: {:3d} | Train loss: {:.3f} | Val loss: {:.3f}".format(
                epoch, mean_train_loss, mean_val_loss
            )
        )

        loss_log.write("{} {} {}\n".format(epoch, train_loss, val_loss))
        loss_log.flush()

        train_losses += train_loss
        val_losses += val_loss

        mean_train_losses.append(mean_train_loss)
        mean_val_losses.append(mean_val_loss)

        save_loss_plot(mean_train_losses, mean_val_losses, opts)

    print("Obtained lowest validation loss of: {}".format(best_val_loss))
    return (train_losses, mean_val_losses)


def train(opts):
    line_pairs, vocab_size, idx_dict = load_data(opts["data_file_name"])
    print_data_stats(line_pairs, vocab_size, idx_dict)

    # Split the line pairs into an 80% train and 20% val split
    num_lines = len(line_pairs)
    num_train = int(0.8 * num_lines)
    train_pairs, val_pairs = line_pairs[:num_train], line_pairs[num_train:]

    # Group the data by the lengths of the source and target words, to form batches
    train_dict = create_dict(train_pairs)
    val_dict = create_dict(val_pairs)

    ##########################################################################
    ### Setup: Create Encoder, Decoder, Learning Criterion, and Optimizers ###
    ##########################################################################
    encoder = TransformerEncoder(
        vocab_size=vocab_size,
        hidden_size=opts.hidden_size,
        num_layers=opts.num_transformer_layers,
        opts=opts,
    )

    decoder = TransformerDecoder(
        vocab_size=vocab_size,
        hidden_size=opts.hidden_size,
        num_layers=opts.num_transformer_layers,
    )

    if opts.cuda:
        encoder.cuda()
        decoder.cuda()
        print("Moved models to GPU!")

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(
        list(encoder.parameters()) + list(decoder.parameters()), lr=opts.learning_rate
    )

    try:
        losses = training_loop(
            train_dict, val_dict, idx_dict, encoder, decoder, criterion, optimizer, opts
        )
    except KeyboardInterrupt:
        print("Exiting early from training.")
        return encoder, decoder, losses

    return encoder, decoder, losses

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads

        assert self.head_dim * num_heads == d_model, "d_model must be divisible by num_heads"

        self.Q = nn.Linear(d_model, d_model)
        self.K = nn.Linear(d_model, d_model)
        self.V = nn.Linear(d_model, d_model)
        self.fc_out = nn.Linear(d_model, d_model)

        self.softmax = nn.Softmax(dim=1)    # Changed from dim=2 to dim=1
        self.scaling_factor = torch.rsqrt(
            torch.tensor(self.hidden_size, dtype=torch.float)
        )

    def forward(self, query, key, value):
        batch_size = query.shape[0]

        query = self.Q(query).view(batch_size, -1, self.num_heads, self.head_dim)
        key = self.K(key).view(batch_size, -1, self.num_heads, self.head_dim)
        value = self.V(value).view(batch_size, -1, self.num_heads, self.head_dim)

        query = query.permute(0, 2, 1, 3)
        key = key.permute(0, 2, 1, 3)
        value = value.permute(0, 2, 1, 3)

        energy = torch.matmul(query, key.permute(0, 1, 3, 2)) / self.head_dim**0.5

        attention = torch.softmax(energy, dim=-1)
        out = torch.matmul(attention, value).permute(0, 2, 1, 3).contiguous()
        out = out.view(batch_size, -1, self.d_model)
        out = self.fc_out(out)

        return out

In [ ]:
class TransformerEncoder(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_layers, opts):
        super(TransformerEncoder, self).__init__()

        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.opts = opts

        self.embedding = nn.Embedding(vocab_size, hidden_size)

        self.self_attentions = nn.ModuleList(
            [
                ScaledDotAttention(
                    hidden_size=hidden_size,
                )
                for i in range(self.num_layers)
            ]
        )
        self.attention_mlps = nn.ModuleList(
            [
                nn.Sequential(
                    nn.Linear(hidden_size, hidden_size),
                    nn.ReLU(),
                )
                for i in range(self.num_layers)
            ]
        )

        self.positional_encodings = self.create_positional_encodings()

    def forward(self, inputs):
        """Forward pass of the encoder RNN.

        Arguments:
            inputs: Input token indexes across a batch for all time steps in the sequence. (batch_size x seq_len)

        Returns:
            annotations: The hidden states computed at each step of the input sequence. (batch_size x seq_len x hidden_size)
            None: Used to conform to standard encoder return signature.
            None: Used to conform to standard encoder return signature.
        """
        batch_size, seq_len = inputs.size()

        encoded = self.embedding(inputs)  # batch_size x seq_len x hidden_size

        # Add positinal embeddings from self.create_positional_encodings. (a'la https://arxiv.org/pdf/1706.03762.pdf, section 3.5)
        encoded = encoded + self.positional_encodings[:seq_len]

        annotations = encoded
        for i in range(self.num_layers):
            new_annotations, self_attention_weights = self.self_attentions[i](
                annotations, annotations, annotations
            )  # batch_size x seq_len x hidden_size
            residual_annotations = annotations + new_annotations
            new_annotations = self.attention_mlps[i](residual_annotations)
            annotations = residual_annotations + new_annotations

        # Transformer encoder does not have a last hidden or cell layer.
        return annotations, None
        # return annotations, None, None
    def create_positional_encodings(self, max_seq_len=1000):
        """Creates positional encodings for the inputs.

        Arguments:
            max_seq_len: a number larger than the maximum string length we expect to encounter during training

        Returns:
            pos_encodings: (max_seq_len, hidden_dim) Positional encodings for a sequence with length max_seq_len.
        """
        pos_indices = torch.arange(max_seq_len)[..., None]
        dim_indices = torch.arange(self.hidden_size // 2)[None, ...]
        exponents = (2 * dim_indices).float() / (self.hidden_size)
        trig_args = pos_indices / (10000**exponents)
        sin_terms = torch.sin(trig_args)
        cos_terms = torch.cos(trig_args)

        pos_encodings = torch.zeros((max_seq_len, self.hidden_size))
        pos_encodings[:, 0::2] = sin_terms
        pos_encodings[:, 1::2] = cos_terms

        if self.opts.cuda:
            pos_encodings = pos_encodings.cuda()

        return pos_encodings